table of contents
1. tips

Juliaの特徴
* コンパイル言語に比肩する実行速度
* シンプルな言語機能と豊富な標準ライブラリ
* 引数の型により実行される関数が決まる動的ディスパッチ
* Lispのような強力なマクロ機能

## 構文糖衣

ここではJuliaで見られる特徴的な構文をざっと見て行きます。

### 係数

変数や関数の前に数値を置くことで、その値との積を表現できます。

In [1]:
x = 2.1

2.1

In [2]:
2x

4.2

In [3]:
4x^2 + 3x - 2

21.94

In [4]:
x = range(0, stop=1, length=50);

In [5]:
4sin.(2x) - 3cos.(4x)

50-element Vector{Float64}:
 -3.0
 -2.8267897373141215
 -2.6339373233305805
 -2.4219133558638326
 -2.1913188324640127
 -1.9428824987789626
 -1.677457339318942
 -1.3960162317568687
 -1.0996467914829426
 -0.7895454385442837
 -0.4670107243016668
 -0.13343596009188374
  0.20969880513734385
  ⋮
  6.996871635711436
  6.996458834488029
  6.969421660503003
  6.9159064004967
  6.836202476105568
  6.730740627863947
  6.60009021313547
  6.444955635692603
  6.266171930354854
  6.064699531632904
  5.841618260670547
  5.598120569893563

### 内包表記

`[]`内に`for`ループを書き、ベクトルや行列などの配列を作ることができます。

In [6]:
[x for x in 1:4]

4-element Vector{Int64}:
 1
 2
 3
 4

In [7]:
[x * y for x in 1:4, y in 1:5]

4×5 Matrix{Int64}:
 1  2   3   4   5
 2  4   6   8  10
 3  6   9  12  15
 4  8  12  16  20

最後に`if`をつけることでフィルターをかけることもできます。

In [8]:
[x for x in 1:10 if isodd(x)]

5-element Vector{Int64}:
 1
 3
 5
 7
 9

辞書も可能です。

In [9]:
Dict(c => i for (i, c) in enumerate('a':'z'))

Dict{Char, Int64} with 26 entries:
  'n' => 14
  'f' => 6
  'w' => 23
  'd' => 4
  'e' => 5
  'o' => 15
  'h' => 8
  'j' => 10
  'i' => 9
  'k' => 11
  'r' => 18
  's' => 19
  't' => 20
  'q' => 17
  'y' => 25
  'a' => 1
  'c' => 3
  'p' => 16
  'm' => 13
  'z' => 26
  'g' => 7
  'v' => 22
  'l' => 12
  'u' => 21
  'x' => 24
  'b' => 2

### 転置

行列を転置するにはシングルクォート(`'`)を末尾につけるだけです。

In [10]:
x = [1 2 3; 4 5 6]

2×3 Matrix{Int64}:
 1  2  3
 4  5  6

In [11]:
x'

3×2 adjoint(::Matrix{Int64}) with eltype Int64:
 1  4
 2  5
 3  6

`transpose`関数でも転置できます。

In [12]:
transpose(x)

3×2 transpose(::Matrix{Int64}) with eltype Int64:
 1  4
 2  5
 3  6

### 無名関数

一時的に使う関数などを、名前を付けずに作ることができます。

In [13]:
x -> 2x

#5 (generic function with 1 method)

In [14]:
map(x -> 2x, [1,2,3])

3-element Vector{Int64}:
 2
 4
 6

### ブロック引数

第一引数に関数を取れる関数では、`do ... end`を使うことで無名関数を後置することもできます。

In [15]:
map([1,2,3]) do x
    2x
end

3-element Vector{Int64}:
 2
 4
 6

### 非標準文字列 (non-standard string literals)

`"..."`のような文字列リテラルの前に識別子を起き、その文字列を元に様々な操作を行うことができます。
以下は、`r"..."`で正規表現オブジェクトを作る例です。

In [17]:
r"\w-\d \w+"

r"\w-\d \w+"

In [18]:
match(r"\w-\d \w+", "B-2 Spirit")

RegexMatch("B-2 Spirit")

バージョン番号を作るのにも使われます。

In [19]:
v"1.2.3"

v"1.2.3"

### 外部コマンド

バッククウォートを使って外部コマンドを作成し、`run`関数で実行できます。

In [20]:
`ls -la`

`ls -la`

In [21]:
run(`ls -la`)

total 1392
drwxr-xr-x  7 yamamuros83  staff     224  5  2 14:34 .
drwxr-xr-x  9 yamamuros83  staff     288  5  2 13:08 ..
drwxr-xr-x  5 yamamuros83  staff     160  5  2 14:28 .ipynb_checkpoints
-rw-r--r--  1 yamamuros83  staff   46831  5  2 14:16 1-section-basic.ipynb
-rw-r--r--  1 yamamuros83  staff  576346  5  2 13:12 1-section.ipynb
-rw-r--r--  1 yamamuros83  staff   30676  5  2 14:28 2-section-function.ipynb
-rw-r--r--  1 yamamuros83  staff   51279  5  2 14:34 3-section-tips.ipynb


Process(`ls -la`, ProcessExited(0))

## メタプログラミング

Juliaには、Juliaのプログラムを使ってJuliaのコードを書き換えることができる**メタプログラミング**という機能があります。JuliaではLisp言語の影響から、この機能を**マクロ**とも呼んでいます。
これを使うと、書くコードの量を劇的に減らしたり、普通の関数では実現できないような特異な働きをさせることができます。

Lispなどでは一般的に使われるマクロですが、他の言語にはあまり見られず、あったとしても自分で定義して使うことは稀でしょう。このチュートリアルでは既に定義されているマクロを使用してみて、マクロではどのようなことが可能になるのかを見ることにします。

### `@show`

`@show`マクロは、変数の値を確認する際に大変便利なマクロです。
これを使うと、コードのある部分で変数が何だったのかを瞬時に知ることができます。

In [22]:
x = 3.14

3.14

In [23]:
@show x

x = 3.14


3.14

挿入ソート(insertion sort)の動作を確認するために、`@show`マクロを使ってみましょう。

In [24]:
insertionsort(xs) = insertionsort!(copy(xs))

function insertionsort!(xs)
    @show xs
    for i in 2:length(xs)
        j = i
        while j > 1 && xs[j-1] > xs[j]
            xs[j-1], xs[j] = xs[j], xs[j-1]
            j -= 1
        end
        @show xs
    end
    xs
end

insertionsort! (generic function with 1 method)

これを実行してみると、ベクトルが左から順にソートされていく様子がよくわかるのではないでしょうか。

In [25]:
insertionsort([6,5,3,1,8,7,2,4])

xs = [6, 5, 3, 1, 8, 7, 2, 4]
xs = [5, 6, 3, 1, 8, 7, 2, 4]
xs = [3, 5, 6, 1, 8, 7, 2, 4]
xs = [1, 3, 5, 6, 8, 7, 2, 4]
xs = [1, 3, 5, 6, 8, 7, 2, 4]
xs = [1, 3, 5, 6, 7, 8, 2, 4]
xs = [1, 2, 3, 5, 6, 7, 8, 4]
xs = [1, 2, 3, 4, 5, 6, 7, 8]


8-element Vector{Int64}:
 1
 2
 3
 4
 5
 6
 7
 8

### `@assert`

`@assert`マクロは、簡便な不定条件のチェックに便利なマクロです。

In [26]:
x = 1

1

条件が満たされるときは、何もしませんが、

In [27]:
@assert x == 1

条件に違反すると、例外を投げます。

In [28]:
@assert x == 2

LoadError: AssertionError: x == 2

エラーメッセージは自分で書くこともできます。

In [29]:
@assert x == 2 "xが$(x)ですよ！ xは2じゃないとダメです！"

LoadError: AssertionError: xが1ですよ！ xは2じゃないとダメです！

### `@time`

`@time`マクロは、関数やコード片の実行時間とメモリ使用量を測るのに便利なマクロです。

先ほど定義した`insertionsort`の`@show`を消して、`quicksort`とパフォーマンスを比較してみましょう。

In [34]:
quicksort(xs) = quicksort!(copy(xs))
quicksort!(xs) = quicksort!(xs, 1, length(xs))

function quicksort!(xs, lo, hi)
    if lo < hi
        p = partition(xs, lo, hi)
        quicksort!(xs, lo, p - 1)
        quicksort!(xs, p + 1, hi)
    end
    return xs
end

function partition(xs, lo, hi)
    pivot = div(lo + hi, 2)
    pvalue = xs[pivot]
    xs[pivot], xs[hi] = xs[hi], xs[pivot]
    j = lo
    @inbounds for i in lo:hi-1
        if xs[i] <= pvalue
            xs[i], xs[j] = xs[j], xs[i]
            j += 1
        end
    end
    xs[j], xs[hi] = xs[hi], xs[j]
    return j
end

partition (generic function with 1 method)

In [35]:
insertionsort(xs) = insertionsort!(copy(xs))

function insertionsort!(xs)
    #@show xs
    @inbounds for i in 2:length(xs)
        j = i
        while j > 1 && xs[j-1] > xs[j]
            xs[j-1], xs[j] = xs[j], xs[j-1]
            j -= 1
        end
        #@show xs
    end
    xs
end

insertionsort([1.0, 0.0])

2-element Vector{Float64}:
 0.0
 1.0

小さい配列だと`insertionsort`と`quicksort`が同程度に速いようですが、

In [36]:
xs = randn(10);

In [37]:
GC.gc()
@time for _ in 1:100000; insertionsort(xs); end

  0.009482 seconds (100.00 k allocations: 15.259 MiB)


In [38]:
GC.gc()
@time for _ in 1:100000; quicksort(xs); end

  0.031742 seconds (115.52 k allocations: 16.127 MiB, 59.03% compilation time)


大きい配列では`quicksort`の方が断然高速です。

In [39]:
xs = randn(10000);

In [40]:
GC.gc()
@time insertionsort(xs);

  0.043568 seconds (2 allocations: 78.203 KiB)


In [41]:
GC.gc()
@time quicksort(xs);

  0.000828 seconds (2 allocations: 78.203 KiB)


## 数値計算

Juliaを使う人の多くは、数値計算を目的としていると思います。
ここで、標準ライブラリで使える便利な関数を紹介していきます。

### 配列の確保

`[1,2,3]`のようにリテラルで直接配列を確保する以外にも、配列を確保する便利な関数が多数あります。

`0`で埋められた配列が欲しい場合は、`zeros`を使います。
基本的には、`zeros(<型>, <サイズ>)`という書き方をします。

In [42]:
x = zeros(Float64, 4)

4-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0

In [43]:
x = zeros(Float64, (4, 3))

4×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [44]:
x = zeros(Float64, (4, 3, 2))

4×3×2 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

同様に`1`で埋められた配列が欲しい場合は`ones`です。

In [45]:
ones(Float64, 4)

4-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0

LinearAlgebraモジュールにある`I`という定数は，単位行列のように振る舞います。

In [46]:
using LinearAlgebra
zeros(3, 3) + 4I

3×3 Matrix{Float64}:
 4.0  0.0  0.0
 0.0  4.0  0.0
 0.0  0.0  4.0

もうちょっと複雑な初期化をしたい場合は、内包表記を使うのが便利でしょう。

In [47]:
[Float64(i > j) for i in 1:4, j in 1:3]

4×3 Matrix{Float64}:
 0.0  0.0  0.0
 1.0  0.0  0.0
 1.0  1.0  0.0
 1.0  1.0  1.0

特に初期化が必要ない場合は、`Array`のコンストラクタを呼ぶこともできます。

In [48]:
Array{Float64}(undef, (3, 4))

3×4 Matrix{Float64}:
 2.22289e-314  2.22289e-314  2.22478e-314  2.22478e-314
 2.22289e-314  2.22478e-314  2.22478e-314  2.22289e-314
 2.22289e-314  2.22478e-314  2.22478e-314  2.21151e-314

`Float64`型はよく使うので、省略可能です。

In [49]:
ones((3, 4))

3×4 Matrix{Float64}:
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0

### 線形代数

ベクトルや配列の和や積は、数値の時のように行えます。

In [50]:
x = [1.0, 2.0, 3.0]

3-element Vector{Float64}:
 1.0
 2.0
 3.0

In [51]:
x .+ 1.0

3-element Vector{Float64}:
 2.0
 3.0
 4.0

In [177]:
x .- 1.0

3-element Array{Float64,1}:
 0.0
 1.0
 2.0

In [52]:
x * 5

3-element Vector{Float64}:
  5.0
 10.0
 15.0

In [179]:
5x

3-element Array{Float64,1}:
  5.0
 10.0
 15.0

In [53]:
x / 2

3-element Vector{Float64}:
 0.5
 1.0
 1.5

ベクトルどうしの和も計算できます。

In [54]:
x + 4x

3-element Vector{Float64}:
  5.0
 10.0
 15.0

In [182]:
0.3 * (x .- 1) + 0.7 * (x .+ 1)

3-element Array{Float64,1}:
 1.4               
 2.3999999999999995
 3.4               

ひとつ気をつけなければいけないことは、ベクトルの要素ごとの積(アダマール積)は、`.*`を使わなければならないということです。これは、`*`がベクトルや行列自体の積に使われているためです。

In [55]:
x * 2x  # 2つの列ベクトルの積は定義されていない

LoadError: MethodError: no method matching *(::Vector{Float64}, ::Vector{Float64})
[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:560
[0m  *([91m::StridedMatrix{T}[39m, ::StridedVector{S}) where {T<:Union{Float32, Float64, ComplexF32, ComplexF64}, S<:Real} at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/matmul.jl:44
[0m  *(::StridedVecOrMat{T} where T, [91m::Adjoint{var"#s832", var"#s831"} where {var"#s832", var"#s831"<:LinearAlgebra.LQPackedQ}[39m) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/lq.jl:254
[0m  ...

In [56]:
x .* 2x  # 2つの列ベクトルの要素毎の積は定義されている

3-element Vector{Float64}:
  2.0
  8.0
 18.0

要素ごとの除算も同様です。

In [57]:
2x ./ x

3-element Vector{Float64}:
 2.0
 2.0
 2.0

内積

In [58]:
dot(x, x)

14.0

行列とベクトルの積

In [59]:
A = [Float64(i + j) for i in 1:2, j in 1:3]

2×3 Matrix{Float64}:
 2.0  3.0  4.0
 3.0  4.0  5.0

In [60]:
A * x

2-element Vector{Float64}:
 20.0
 26.0

行列の積

In [61]:
A' * A

3×3 Matrix{Float64}:
 13.0  18.0  23.0
 18.0  25.0  32.0
 23.0  32.0  41.0

行列の積は次のように`*`を省略しても書けます。

In [62]:
A'A

3×3 Matrix{Float64}:
 13.0  18.0  23.0
 18.0  25.0  32.0
 23.0  32.0  41.0

一次線形方程式 $A x = b$ の解 $x$

In [63]:
b = [20.0, 26.0]
A \ b

3-element Vector{Float64}:
 1.0000000000000142
 2.000000000000001
 2.9999999999999902

## モジュール

すべての名前が一つの場所にあると、関数名などが衝突して予期せぬ動作をすることがあります。
特に、Juliaのように多重ディスパッチがあるときは問題が複雑になりかねません。
そういった問題を回避するためにも、名前空間を分けるJuliaのモジュールシステムを理解することは重要です。

Juliaでは、モジュールは`module ... end`で明示的に定義します。
`module`と`end`で囲まれた部分のコードは、他の名前空間とは別の名前空間になります。
つまり、型や関数名は他のモジュールと同じ名前であっても衝突しません。

In [64]:
module ModFoo

struct Foo
    x
end

function show(x)
    print("!! " * string(x) * " !!")
end

end

Main.ModFoo

あるモジュール内の型や関数にアクセスするときは、`<モジュール名>.<型/関数名>`というようにアクセスします。

In [65]:
f = ModFoo.Foo("something")

Main.ModFoo.Foo("something")

このとき、関数`show`は元々用意されている関数`show`とは衝突していません。

In [66]:
ModFoo.show(f)

!! Main.ModFoo.Foo("something") !!

In [67]:
show(f)

Main.ModFoo.Foo("something")

では、`ModFoo`というモジュール名自体にはどのようにアクセスしているのでしょうか。
Juliaには、デフォルトのモジュールが既に用意されており、`Main`と呼ばれています。
今まで書いてきた変数・関数・モジュールはすべてこの`Main`モジュールに紐付けられており、ここを探索しているというわけです。

In [68]:
Main

Main

In [69]:
Main.quicksort([3,1,2])

3-element Vector{Int64}:
 1
 2
 3

In [70]:
Main.f

Main.ModFoo.Foo("something")

In [71]:
Main.ModFoo

Main.ModFoo

### モジュールの使い方

以下の様なモジュール`Geo`を定義したとしましょう。

In [72]:
module Geo

export Point, distance, iswithin, move

struct Point
    x::Float64
    y::Float64
    z::Float64
end

distance(a::Point, b::Point) = sqrt(sqdist(a, b))

iswithin(p::Point, c::Point, r::Real) = sqdist(p, c) <= r^2

function move(p::Point, dx, dy, dz)
    return Point(p.x + dx, p.y + dy, p.z + dz)
end

sqdist(a::Point, b::Point) = (a.x - b.x)^2 + (a.y - b.y)^2 + (a.z - b.z)^2

end

Main.Geo

このモジュールで定義されている型や関数を、現在のモジュール(`Main`)で使いたいときは、`using`を使います。

In [73]:
using .Geo

`using`を使うと、`export`で指定された名前が`Geo.`をつけなくても利用可能になります。

In [74]:
po = Point(0, 0, 0)
p1 = Point(1, 1, 1)

Point(1.0, 1.0, 1.0)

In [75]:
distance(po, p1)

1.7320508075688772

In [76]:
iswithin(p1, po, 1.7)

false

In [77]:
move(p1, -1, 0, 0)

Point(0.0, 1.0, 1.0)

In [78]:
iswithin(p1, po, 1.7)

false

しかし、使える関数は3行目の`export`で指定されたものだけです。`sqdist`は`export`に指定されていないため使えません。

In [79]:
sqdist(po, p1)

LoadError: UndefVarError: sqdist not defined

`export`されていない場合は、モジュール名もつける必要があります。

In [80]:
Geo.sqdist(po, p1)

3.0

### パッケージ

人の作ったモジュールを使うことで、開発の時間は短縮できます。
再利用を目的としたモジュールの集まりは、ひとつのパッケージにまとめられGitHubを中心に配布されています。
パッケージは`METADATA`というレポジトリに集められ、JuliaのREPLから`Pkg.add`関数を呼ぶだけで簡単にインストールできます。

パッケージを探すときは、<http://pkg.julialang.org/>を参照してみてください。

ここでは、[Optim.jl](https://github.com/JuliaOpt/Optim.jl)パッケージを利用して、関数の数値最適化をしてみましょう。
まずはインストールです。

In [209]:
# 最初に実行するときだけ、下のコメントを消して実行して下さい
using Pkg
Pkg.update()
Pkg.add("Optim")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating git-repo `git@github.com:bicycle1885/CellFishing.jl.git`
 Resolving package versions...
  Updating `~/.julia/environments/v0.7/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v0.7/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v0.7/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v0.7/Manifest.toml`
 [no changes]


In [210]:
using Optim  # インストール直後の読み込みはやや時間がかかる

[Rosenbrock関数](http://en.wikipedia.org/wiki/Rosenbrock_function)を最適化してみましょう。
この関数は以下のように定義され、 $(x, y) = (a, a^2)$ で最小値をとります。

$$f(x, y) = (a-x)^2 + b(y-x^2)^2$$

In [211]:
func(x, a=1, b=100) = (a - x[1])^2 + b * (x[2] - x[1]^2)^2

func (generic function with 3 methods)

Optim.jlでは、`optimize`関数を`export`しており、Nelder-Meadアルゴリズムを使って、実際の最小解 $(1, 1)$ をほぼ達成できています。

In [212]:
optimize(func, [10.0, 10.0])

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [10.0,10.0]
 * Minimizer: [1.0000076575345525,1.0000090769441363]
 * Minimum: 3.950131e-09
 * Iterations: 103
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 198

---

In [81]:
versioninfo()

Julia Version 1.6.1
Commit 6aaedecc44 (2021-04-23 05:59 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.7.0)
  CPU: Intel(R) Core(TM) i5-7267U CPU @ 3.10GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, skylake)
